# Deep Network for Design Parameters for FWUAV

## Importing the Modules

In [1]:
# Modules to import for data preprocessing
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Modules to import for Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Check for GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Loading the Data

In [3]:
# Loading the data into the pandas dataframe

# Printing the datafram 
print("\n Data loaded into pandas : \n", data)

# Reading the Data
data = pd.read_csv("Design_Data.csv", header=None)
print("dataLoaded :", data)
# Set the first column to object type
data[:0] = data[:0].astype('object')

# Set dtype for the rest of the columns to float32
data.iloc[:, 1:] = data.iloc[:, 1:].astype('float32')

print("\n---------------------------------------------------------------------------\n")

# Inserting the data columns 
data.columns = [
    'Airfoil','Wing Span', 'Taper Ratio', 'Aspect Ratio', 'Flapping Period',
    'Airspeed', 'Angle of Attack', 'Lift', 'Induced Drag'
]

print("\n---------------------------------------------------------------------------\n")

# Split the data into features and targets
X = data[[
    'Airfoil','Wing Span', 'Taper Ratio', 'Aspect Ratio', 'Flapping Period',
    'Airspeed', 'Angle of Attack'
]]

print("Features : \n", X.values)

print("\n---------------------------------------------------------------------------\n")
y = data[['Lift', 'Induced Drag']]

print("Targets : \n",y.values)




 Data loaded into pandas : 
 <module 'torch.utils.data' from '/home/stimp/anaconda3/envs/mlenv/lib/python3.8/site-packages/torch/utils/data/__init__.py'>
dataLoaded :               0    1    2    3    4    5   6          7         8
0      naca8304  0.4  1.5  0.2  0.4  3.0   5  -0.185762  0.010605
1      naca8304  0.4  1.5  0.2  0.4  3.0  15  -0.496471 -0.053405
2      naca8304  0.4  1.5  0.2  0.4  3.0  25  -0.824951 -0.161062
3      naca8304  0.4  1.5  0.2  0.4  3.0  35  -1.162489 -0.325510
4      naca8304  0.4  1.5  0.2  0.4  4.0   5  -0.319926  0.002657
...         ...  ...  ...  ...  ...  ...  ..        ...       ...
10682  naca2412  0.8  2.0  0.4  1.1  6.5  30 -26.595298 -3.207132
10683  naca0012  0.8  2.0  0.4  1.1  6.5  30 -25.542093 -2.913544
10684    goe225  0.8  2.0  0.4  1.1  6.5  35 -33.382168 -5.573355
10685  naca2412  0.8  2.0  0.4  1.1  6.5  35 -30.313721 -4.444688
10686  naca0012  0.8  2.0  0.4  1.1  6.5  35 -29.279326 -4.102922

[10687 rows x 9 columns]

-------------

## One Hot encoder and scaler fit


In [4]:
# Assuming X and y are already defined in your Jupyter notebook
# X - Features DataFrame
# y - Targets DataFrame

# Drop the Airfoil column from the features
airfoils = X['Airfoil']

# One hot encode the airfoil column
encoder = OneHotEncoder(sparse_output=False)
encoded_airfoil = encoder.fit_transform(airfoils.values.reshape(-1, 1))

print("\nEncoded Airfoils:\n", encoded_airfoil)

X_num = X.drop('Airfoil', axis=1)

Scalerx = MinMaxScaler()
Scalery = MinMaxScaler()

X_scaled_np = Scalerx.fit_transform(X_num)
X_scaled = pd.DataFrame(X_scaled_np, columns=X_num.columns)
print("\nNormalized Features without Airfoil:\n", X_scaled)

y_scaled_np = Scalery.fit_transform(y.values)
y_scaled = pd.DataFrame(y_scaled_np, columns=y.columns)
print("\nNormalized Targets:\n", y_scaled)

X_encoded_pd = pd.DataFrame(encoded_airfoil, columns=encoder.get_feature_names_out(['Airfoil']))
print("\nEncoded Airfoil DataFrame:\n", X_encoded_pd)

X_final = pd.concat([X_encoded_pd, X_scaled], axis=1)
print("\nScaled and Encoded Features:\n", X_final)



Encoded Airfoils:
 [[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

Normalized Features without Airfoil:
        Wing Span  Taper Ratio  Aspect Ratio  Flapping Period  Airspeed  \
0       0.076923     0.076923      0.000000         0.222222  0.222222   
1       0.076923     0.076923      0.000000         0.222222  0.222222   
2       0.076923     0.076923      0.000000         0.222222  0.222222   
3       0.076923     0.076923      0.000000         0.222222  0.222222   
4       0.076923     0.076923      0.000000         0.222222  0.444444   
...          ...          ...           ...              ...       ...   
10682   0.384615     0.230769      0.333333         1.000000  1.000000   
10683   0.384615     0.230769      0.333333         1.000000  1.000000   
10684   0.384615     0.230769      0.333333         1.000000  1.000000   
10685   0.384615     0.230769      0.333333         1.000000  1.000000   
10686   0.384615     0.230769  

# Splitting the data into training and testing 

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y_scaled, test_size=0.2, random_state=42)

# Verify the split
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)


Training features shape: (8549, 10)
Testing features shape: (2138, 10)
Training target shape: (8549, 2)
Testing target shape: (2138, 2)


# Defining the Model

In [6]:

class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(10, 128)  # Input layer
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(128, 128)  # New layer
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(0.5)  # Dropout layer
        self.fc4 = nn.Linear(64, 64)  # New layer
        self.dropout4 = nn.Dropout(0.5)  # Dropout layer
        self.fc5 = nn.Linear(64, 32)
        self.dropout5 = nn.Dropout(0.5)  # Dropout layer
        self.fc6 = nn.Linear(32, 16)  # New layer
        self.dropout6 = nn.Dropout(0.5)  # Dropout layer
        self.fc7 = nn.Linear(16, 2)    # Output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.relu(self.fc4(x))
        x = self.dropout4(x)
        x = self.relu(self.fc5(x))
        x = self.dropout5(x)
        x = self.relu(self.fc6(x))
        x = self.dropout6(x)
        x = self.fc7(x)
        return x



# class DeepNN(nn.Module):
#     def __init__(self):
#         super(DeepNN, self).__init__()
#         self.fc1 = nn.Linear(10, 128)  # Change input features to 10
#         self.dropout1 = nn.Dropout(0.5)  # Dropout layer
#         self.fc2 = nn.Linear(128, 64)
#         self.dropout2 = nn.Dropout(0.5)  # Dropout layer
#         self.fc3 = nn.Linear(64, 32)
#         self.dropout3 = nn.Dropout(0.5)  # Dropout layer
#         self.fc4 = nn.Linear(32, 2)    # Change output features to 2
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.dropout1(x)
#         x = self.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.relu(self.fc3(x))
#         x = self.dropout3(x)
#         x = self.fc4(x)
#         return x


In [7]:
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

In [ ]:
print(X_train.shape)
print(y_test)

torch.Size([8549, 10])
tensor([[0.9192, 0.1097],
        [0.9863, 0.0346],
        [0.7682, 0.2483],
        ...,
        [0.9631, 0.0365],
        [0.9796, 0.0491],
        [0.9909, 0.0391]], device='cuda:0')


# Training and eval of the model 

In [11]:
# Move the model to GPU if available
model = DeepNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping
patience = 1000
best_loss = float('inf')
best_model_state = None
no_improvement_epochs = 0

# Training the model
num_epochs = 100000000
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    train_loss = criterion(outputs, y_train)
    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)

    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

    if test_loss < best_loss:
        best_loss = test_loss
        best_model_state = model.state_dict()
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}')

# Load the best model state
if best_model_state is not None:
    model.load_state_dict(best_model_state)


Epoch [10/100000000], Train Loss: 0.5359395146369934, Test Loss: 0.5152684450149536
Epoch [20/100000000], Train Loss: 0.2992209196090698, Test Loss: 0.14116019010543823
Epoch [30/100000000], Train Loss: 0.25036630034446716, Test Loss: 0.1928849071264267
Epoch [40/100000000], Train Loss: 0.21754753589630127, Test Loss: 0.11031836271286011
Epoch [50/100000000], Train Loss: 0.19263386726379395, Test Loss: 0.12686344981193542
Epoch [60/100000000], Train Loss: 0.17454169690608978, Test Loss: 0.09345362335443497
Epoch [70/100000000], Train Loss: 0.16456258296966553, Test Loss: 0.09136099368333817
Epoch [80/100000000], Train Loss: 0.15258778631687164, Test Loss: 0.08774661272764206
Epoch [90/100000000], Train Loss: 0.14428922533988953, Test Loss: 0.07891284674406052
Epoch [100/100000000], Train Loss: 0.14139726758003235, Test Loss: 0.07089122384786606
Epoch [110/100000000], Train Loss: 0.13477599620819092, Test Loss: 0.06549179553985596
Epoch [120/100000000], Train Loss: 0.12772206962108612, 

# Saving the Model

In [10]:
torch.save(model.state_dict(), 'avg_flightdata_model.pth')

# Loading the saved model and Predicting with it 

In [14]:
# Load the saved model
modelloaded = DeepNN()  
modelloaded.load_state_dict(torch.load('avg_flightdata_model.pth'))
modelloaded.to(device)
modelloaded.eval()

/tmp/ipykernel_79148/3300405255.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelloaded.load_state_dict(torch.load('avg_flightdata_model.pth'))


DeepNN(
  (fc1): Linear(in_features=10, out_features=128, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
)

In [25]:
# Sample input tensor with proper formatting
sample_input = torch.tensor([
    [
        0.0,      # Feature 1
        0.0,      # Feature 2
        0.0,      # Feature 3
        1.0,      # Feature 4
        0.076923, # Feature 5
        0.076923, # Feature 6
        0.000000, # Feature 7
        0.222222, # Feature 8
        0.222222,  # Feature 9
        0.285714  # Feature 10
    ]
], dtype=torch.float32).to(device)

# Make prediction using model
with torch.no_grad():
    prediction = model(sample_input)
    outputs = prediction[0].cpu().numpy()

# Transform predictions back to original scale
transformed_outputs = Scalery.inverse_transform(outputs.reshape(1, -1))[0]

# Print formatted predictions
print(f"Predicted Lift: {transformed_outputs[0]:.4f}")
print(f"Predicted Induced Drag: {transformed_outputs[1]:.4f}")


Predicted Lift: -5.5712
Predicted Induced Drag: 1.4453


In [38]:
# Define input array
input_array = [1, 0, 0, 0, 0.4, 1.5, 0.2, 0.4, 3.0, 5.0]

# Split array into airfoil and other features
airfoil = np.array(input_array[0:4])
other_features = Scalerx.transform(np.array(input_array[4:]).reshape(1, -1))

# Concatenate arrays properly
input_array_concat = np.concatenate((airfoil, other_features.flatten()))

# Convert to PyTorch tensor and move to device
model_input = torch.tensor(input_array_concat.reshape(1, -1), dtype=torch.float32).to(device)

# Make prediction using model
with torch.no_grad():
    prediction = model(model_input)
    outputs = prediction[0].cpu().numpy()

# Transform predictions back to original scale
transformed_outputs = Scalery.inverse_transform(outputs.reshape(1, -1))[0]

# Print formatted predictions
print(f"Predicted Lift: {transformed_outputs[0]:.4f}")
print(f"Predicted Induced Drag: {transformed_outputs[1]:.4f}")

Predicted Lift: -5.4858
Predicted Induced Drag: 1.4588


/home/stimp/anaconda3/envs/pteraenv/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
